In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import geopandas as gpd

In [8]:
# Read CSVs
amphibians_csv = pd.read_csv("ERL_Pulse/Amphibians/European Red List of Amphibians 2025 - all species assessments.csv")
reptiles_csv = pd.read_csv("ERL_Pulse/Bees/European Red List of Bees 2025 - all species assessments.csv")
bees_csv = pd.read_csv("ERL_Pulse/Reptiles/European Red List of Reptiles 2025 - all species assessments.csv")

# Add 'group' and standardized 'species_name' columns
amphibians_csv['group'] = 'amphibian'
reptiles_csv['group'] = 'reptile'
bees_csv['group'] = 'bee'

# Combine CSVs
all_species_csv = pd.concat([amphibians_csv, reptiles_csv, bees_csv], ignore_index=True)
all_species_csv = all_species_csv.rename(columns={'Species name': 'species_name'})
print(all_species_csv.columns)
print(all_species_csv.shape)

Index(['Order', 'Family', 'species_name', 'Taxonomic authority', 'Common name',
       'taxon_id', 'assessment_id', 'European category',
       'EU 27 regional category', 'Endemic to Europe', 'Endemic to EU 27',
       'Link to assessment', 'group'],
      dtype='object')
(2463, 13)


In [11]:
# Standardize species names in the GeoDataFrames
species_name_col = {
    'amphibian': 'SCI_NAME',
    'reptile': 'sci_name',
    'bee': 'sci_name'
}

# Paths for GEM layers
gpkg_paths = {
    "amphibian": "ERL_Pulse/Amphibians/Amphibian_maps_all_geopackage.gpkg",
    "reptile": "ERL_Pulse/Reptiles/Reptile_maps_all_geopackage.gdb.gpkg",
    "bee": "ERL_Pulse/Bees/Bee_maps_all.gpkg"
}

# GEM layer names
gem_layers = {
    "amphibian": "Amphibian_maps_GEM",
    "reptile": "Reptile_maps_GEM",
    "bee": "Bee_maps_GEM"
}

# Read GEM layers, convert CRS to common CRS (WGS84 EPSG:4326), add group
gdfs = []
for group, path in gpkg_paths.items():
    gdf = gpd.read_file(path, layer=gem_layers[group])
    gdf = gdf.to_crs(epsg=4326)  # convert to WGS 84
    gdf['group'] = group
    # Add standardized species_name column
    gdf['species_name'] = gdf[species_name_col[group]]
    gdfs.append(gdf)

# Concatenate all GeoDataFrames
all_species_gdf = pd.concat(gdfs, ignore_index=True)
print(all_species_gdf.columns)
print(all_species_gdf.shape)

Index(['SCI_NAME', 'presence', 'origin', 'seasonal', 'compiler', 'citation',
       'source', 'dist_comm', 'island', 'subspecies', 'subpop', 'legend',
       'tax_comm', 'yrcompiled', 'data_sens', 'sens_comm', 'generalisd',
       'id_no', 'Shape_Leng', 'ASSESSMENT', 'Sci_name_1', 'geometry', 'group',
       'species_name', 'sci_name', 'PRESENCE', 'ORIGIN', 'SEASONAL',
       'assessment'],
      dtype='object')
(290, 29)


In [12]:
merged_gdf = all_species_gdf.merge(
    all_species_csv,
    on=['species_name', 'group'],
    how='left'
)

In [13]:
# Columns to always keep
base_cols = ['species_name', 'group', 'geometry', 'Order', 'Family', 
             'Common name', 'European category', 'EU 27 regional category', 
             'Endemic to Europe', 'Endemic to EU 27']

# Columns that may have duplicates
duplicate_cols = ['assessment', 'presence', 'origin', 'seasonal']

# Start cleaned GeoDataFrame
cleaned_gdf = merged_gdf[base_cols].copy()

# For each potentially duplicated column, combine duplicates by taking first non-null
for col in duplicate_cols:
    cols_to_merge = [c for c in merged_gdf.columns if c.lower() == col.lower()]
    cleaned_gdf[col] = merged_gdf[cols_to_merge].bfill(axis=1).iloc[:, 0]

In [14]:
print(cleaned_gdf.columns)
print(cleaned_gdf.shape)
cleaned_gdf[cleaned_gdf.isnull().any(axis=1)]

Index(['species_name', 'group', 'geometry', 'Order', 'Family', 'Common name',
       'European category', 'EU 27 regional category', 'Endemic to Europe',
       'Endemic to EU 27', 'assessment', 'presence', 'origin', 'seasonal'],
      dtype='object')
(290, 14)


,species_name,group,geometry,Order,Family,Common name,European category,EU 27 regional category,Endemic to Europe,Endemic to EU 27,assessment,presence,origin,seasonal
6,Bufotes balearicus,amphibian,"MULTIPOLYGON (((12.47481 44.88934, 12.48421 44...",Anura,Bufonidae,NaN,LC,LC,Yes,Yes,228180449.0,1.0,1.0,1.0
7,Bufotes balearicus,amphibian,"MULTIPOLYGON (((3.17381 39.95884, 3.16871 39.9...",Anura,Bufonidae,NaN,LC,LC,Yes,Yes,228180449.0,1.0,5.0,1.0
53,Acanthodactylus schreiberi,reptile,"MULTIPOLYGON (((34.58851 35.69491, 34.58852 35...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,207990075.0,1.0,1.0,1.0
54,Algyroides fitzingeri,reptile,"MULTIPOLYGON (((9.71336 40.84884, 9.71138 40.8...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,137846377.0,1.0,1.0,1.0
55,Algyroides marchi,reptile,"MULTIPOLYGON (((-2.22447 38.75499, -2.22559 38...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,137842545.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285,Simpanurgus phyllopodus,bee,"MULTIPOLYGON (((-3.87875 40.79182, -3.8683 40....",NaN,NaN,NaN,NaN,NaN,NaN,NaN,222201063.0,4.0,1.0,1.0
286,Sphecodes aetnensis,bee,"MULTIPOLYGON (((15.58918 38.24238, 15.60808 38...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,229923962.0,1.0,1.0,1.0
287,Sphecodes creticus,bee,"MULTIPOLYGON (((23.90707 35.23626, 23.91009 35...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,222216586.0,1.0,1.0,1.0
288,Stelis ortizi,bee,"MULTIPOLYGON (((-5.41732 40.87752, -5.40591 40...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,222214120.0,1.0,1.0,1.0
